# План работы:
- Кроме Videos.md должен быть файл Videos.html - а лучше ipynb - тогда будет встраивание в обоих формах видео, описание в предыдущей markdown-ячейке.
- Для оценки структуры курса и генерации элементов - ячейка с считыванием всех имеющихся блоков, в которых source для Markdown содержит знаки решетки. Это автоматически отправлять в Syllabus
- Сделать взаимодополняющимися разделы "Литература" - Переносить из файла Literature.md в соответствующие разделы и обратно

In [173]:
import os
import json
os.chdir('../course') # Все действия осуществляются из директории курса

from IPython.display import display, Markdown, Latex

# Определение путей всех имеющихся в курсе ipynb-файлов

In [174]:
ipynb_paths = []

for dirname, dirnames, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith('.ipynb') & (not filename.endswith('-checkpoint.ipynb')):
            ipynb_paths.append(os.path.join(dirname, filename))

# Актуальное содержание в READ.me файле

In [175]:
def chapters():
    chs = dict()
    for i, ipynb_path in enumerate(sorted(ipynb_paths)):
        with open(ipynb_path) as f:
            data = json.load(f)
            name = data['cells'][0]['source'][0]  # Заголовок файла это всегда первая ячейка
            markdown_path = os.path.join(os.path.basename(os.path.dirname(ipynb_path)), os.path.basename(ipynb_path))
            # срез так как название начинается с решетки и пробела
            chs[ipynb_path] = '{0}. [{1}]({2})'.format(i+1, name[2:].strip(), markdown_path)
    return chs
            
with open('README.md') as f:
    blocks = f.read().split('#')
    for i, block in enumerate(blocks):
        if block.startswith(' Содержание курса'):
            blocks[i] = ' Содержание курса\n\n' + '\n'.join([chapters()[key] for key in sorted(chapters())]) + '\n\n'
    f.close()

with open('README.md', 'w') as f:           
    f.write("#".join(blocks)) 
    f.close()

# print('Ниже идет текущий вид сгенерированного файла READ.me')
# with open('README.md') as f:
#     display(Markdown(f.read()))
#     f.close()

# Полная структура курса

In [176]:
# Рассматриваем только для markdown cells

# Генерация страницы с полным списком видео

Для удобства просмотра видео на досуге сделаем общую страницу, в которой видеофрагменты из всех Jupyter-блокнотов будут собираться в общий файл [Videos.md]().

In [177]:
md_first_lines = '''
# Видеотека

Данный файл содержит подборку из всех видеофрагментов
курса "Наноматериалы". Этот файл более удобен, если вы
открыли его на GitHub. Однако разметка Markdown не позволяет
встраивать видео. Каждое изображение является ссылкой на 
видеоролик. Если вы открыли курс на компьютере,
советуем воспользоваться [html-файлом](Videos.html).
Это позволит воспроизводить все видео на одной странице.

Приведенные видеофрагменты не являются обязательными
к просмотру, но позволяют увидеть рассматриваемые проблемы
с позиций других исследователей. Некоторые видеоролики
полезны как примеры лабораторных практик по синтезу
и диагностике различных форм наноматериалов.\n\n'''


page = ''

for ipynb_path in sorted(chapters().keys()):
    with open(ipynb_path) as f:
        head = '# {} \n'.format(chapters()[ipynb_path])
        page += head
        data = json.load(f)
        for cell in data['cells']:
            if (cell['cell_type'] == 'code') & ('YouTubeVideo' in ''.join(cell['source'])):
                d = "".join(cell['source'])
                d = d.replace('\n', ' ')
                try:
                    d = d[3:d[d.index("'''")+3:].index("'''")+3]
                    d = d.lstrip()
                except ValueError:
                    d = '**У этого видео пока нет описания.**'
                for line in cell['source']:
                    if "YouTubeVideo('" in line.strip():
                        short_link = line[len("YouTubeVideo('"):-len(")'")]
                image_url = 'https://img.youtube.com/vi/{}/0.jpg'.format(short_link)
                movie_url = 'https://www.youtube.com/watch?v={}'.format(short_link)
                image_block = '[![]({0})]({1})\n'.format(image_url, movie_url)
                movie_with_description = '{0}\n\n {1}\n\n ---- \n'.format(d, image_block)
                page += movie_with_description

page = md_first_lines+page
#display(Markdown(page))

with open('Videos.md', 'w') as f:
    f.write(page)
    f.close()

# Общий объем текстовой части

In [197]:
total_symbols = 0
for ipynb_path in sorted(chapters().keys()):
    with open(ipynb_path) as f:
        data = json.load(f)
        total_symbols += sum([len(''.join(cell['source'])) for cell in data['cells'] if cell['cell_type'] == 'markdown'])

print('Общий объем непрограммного текста - {} символов, что соответствует примерно {} листов А4.'.format(total_symbols, total_symbols//1800))

Общий объем непрограммного текста - 195073 символов, что соответствует примерно 108 листов А4.
